In [ ]:
!pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.5/235.7 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Erstellen des Datensatzes für das Training für das Autobauteil Erkennungs Modell

In [ ]:

"""
Skript zur Vorbereitung des Bauteil-Datensatzes.

"""
from pathlib import Path
import kagglehub
import json
import os
import shutil
import random
import yaml
from tqdm import tqdm

# --- Pfad zum Haupt-Projektordner auf Google Drive ---
GDRIVE_PROJECT_PATH = Path('/content/drive/MyDrive/YOLO_Car_Damage_Project')

PART_CLASSES = ['Windshield', 'Back-windshield', 'Front-window', 'Back-window', 'Front-door', 'Back-door', 'Front-wheel', 'Back-wheel', 'Front-bumper', 'Back-bumper', 'Headlight', 'Tail-light', 'Hood', 'Trunk', 'License-plate', 'Mirror', 'Roof', 'Grille', 'Rocker-panel', 'Quarter-panel', 'Fender']

def prepare_parts_dataset_corrected(validation_split=0.15, test_split=0.15):
    """
    Bereitet den Bauteil-Datensatz korrekt vor und speichert ihn auf Google Drive.
    """
    print("Starte korrigierte Vorbereitung: Reiner Bauteil-Datensatz (70/15/15 Split)...")
    try:
        download_path = Path(kagglehub.dataset_download("humansintheloop/car-parts-and-car-damages"))
        print(f"Datensatz erfolgreich nach '{download_path}' heruntergeladen.")
    except Exception as e:
        print(f"FEHLER beim Herunterladen des Datensatzes: {e}")
        return

    # Zielordner direkt auf Google Drive
    yolo_dir_on_drive = GDRIVE_PROJECT_PATH / 'datasets' / 'yolo_parts_only_dataset'
    print(f"Zielordner auf Google Drive: {yolo_dir_on_drive}")

    if yolo_dir_on_drive.exists():
        print(f"Alter, fehlerhafter Datensatz-Ordner gefunden. Lösche ihn...")
        shutil.rmtree(yolo_dir_on_drive)

    # Pfade zu den Bauteil-Daten
    parts_img_dir = download_path / "Car damages dataset" / "File1" / "img"
    parts_ann_dir = download_path / "Car damages dataset" / "File1" / "ann"

    # Erstelle alle benötigten Unterordner
    for sub in ['images/train', 'images/val', 'images/test', 'labels/train', 'labels/val', 'labels/test']:
        (yolo_dir_on_drive / sub).mkdir(parents=True, exist_ok=True)

    class_to_idx = {name: i for i, name in enumerate(PART_CLASSES)}

    all_files_to_process = []
    for ann_path in tqdm(list(parts_ann_dir.glob('*.json')), desc="Sammle und prüfe Dateien"):
        image_name = ann_path.stem
        image_path = parts_img_dir / image_name
        if image_path.exists():
            all_files_to_process.append((ann_path, image_path))

    random.shuffle(all_files_to_process)

    total_files = len(all_files_to_process)
    val_split_idx = int(total_files * validation_split)
    test_split_idx = val_split_idx + int(total_files * test_split)

    file_splits = {
        "val": all_files_to_process[:val_split_idx],
        "test": all_files_to_process[val_split_idx:test_split_idx],
        "train": all_files_to_process[test_split_idx:]
    }

    # Verarbeite jede Aufteilung
    for subset, files in file_splits.items():
        print(f"Verarbeite {subset}-Set ({len(files)} Bilder)...")
        for ann_path, image_path in tqdm(files, desc=f"Erstelle {subset}-Set"):
            with open(ann_path, 'r') as f: data = json.load(f)
            size_info = data.get('size', {}); image_width, image_height = size_info.get('width'), size_info.get('height')
            if not image_width or not image_height: continue

            yolo_annotations = []
            for obj in data.get('objects', []):
                if obj.get('classTitle') in PART_CLASSES:
                    points = obj.get('points', {}).get('exterior', [])
                    if len(points) > 1:
                        x_coords = [p[0] for p in points]; y_coords = [p[1] for p in points]
                        x_min, y_min, x_max, y_max = min(x_coords), min(y_coords), max(x_coords), max(y_coords)
                        box_width, box_height = x_max - x_min, y_max - y_min
                        x_center, y_center = x_min + box_width / 2, y_min + box_height / 2
                        x_norm, y_norm, w_norm, h_norm = x_center/image_width, y_center/image_height, box_width/image_width, box_height/image_height

                        # --- HIER WAR DER FEHLERURSPRUNG ---
                        # Die korrekte Logik stellt sicher, dass eine NUMMER, nicht der Text, verwendet wird.
                        class_idx = class_to_idx[obj.get('classTitle')]
                        yolo_annotations.append(f"{class_idx} {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}")

            if not yolo_annotations: continue

            shutil.copy(image_path, yolo_dir_on_drive / 'images' / subset / image_path.name)
            with open(yolo_dir_on_drive / 'labels' / subset / f"{image_path.stem}.txt", 'w') as f:
                f.write("\n".join(yolo_annotations))

    # Finale dataset.yaml mit absolutem Pfad für Google Drive erstellen
    yaml_path = yolo_dir_on_drive / 'dataset.yaml'
    idx_to_class = {i: name for i, name in enumerate(PART_CLASSES)}
    yaml_data = {
        'path': str(yolo_dir_on_drive.resolve()),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(PART_CLASSES),
        'names': idx_to_class
    }
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)

    print("\n--- Prozess Abgeschlossen ---")
    print(f"Bauteil-Datensatz fertig in '{yolo_dir_on_drive.resolve()}'.")

if __name__ == '__main__':
    prepare_parts_dataset_corrected()


Starte korrigierte Vorbereitung: Reiner Bauteil-Datensatz (70/15/15 Split)...
Datensatz erfolgreich nach '/kaggle/input/car-parts-and-car-damages' heruntergeladen.
Zielordner auf Google Drive: /content/drive/MyDrive/YOLO_Car_Damage_Project/datasets/yolo_parts_only_dataset
Alter, fehlerhafter Datensatz-Ordner gefunden. Lösche ihn...


Sammle und prüfe Dateien: 100%|██████████| 998/998 [00:01<00:00, 607.74it/s]


Verarbeite val-Set (149 Bilder)...


Erstelle val-Set: 100%|██████████| 149/149 [00:04<00:00, 32.04it/s]


Verarbeite test-Set (149 Bilder)...


Erstelle test-Set: 100%|██████████| 149/149 [00:04<00:00, 30.23it/s]


Verarbeite train-Set (700 Bilder)...


Erstelle train-Set: 100%|██████████| 700/700 [00:23<00:00, 29.88it/s]


--- Prozess Abgeschlossen ---
Bauteil-Datensatz fertig in '/content/drive/MyDrive/YOLO_Car_Damage_Project/datasets/yolo_parts_only_dataset'.


# Training des Autobauteil Erkennungs Modell

In [ ]:

"""
Skript für Modell 1: Der reine Bauteil-Detektor.
"""
import os
import sys
import subprocess
import pkg_resources
import torch
from pathlib import Path
import shutil
from ultralytics import YOLO

# --- Pfad zum Haupt-Projektordner auf Google Drive ---
GDRIVE_PROJECT_PATH = Path('/content/drive/MyDrive/YOLO_Car_Damage_Project')

def train_yolov11_part_detector():
    """
    Trainiert, speichert und evaluiert das YOLOv11m Bauteil-Erkennungsmodell.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Verwende Gerät: {device}")

    if not GDRIVE_PROJECT_PATH.exists():
        print(f"FEHLER: Der Google Drive Projektordner '{GDRIVE_PROJECT_PATH}' wurde nicht gefunden.")
        return

    model = YOLO('yolo11m.pt')

    # --- KORREKTER PFAD: Zeigt auf den Datensatz im Google Drive ---
    # Das Datenvorbereitungs-Skript speichert den Datensatz an diesem Ort.
    dataset_yaml_path = GDRIVE_PROJECT_PATH / 'datasets' / 'yolo_parts_only_dataset' / 'dataset.yaml'

    if not Path(dataset_yaml_path).exists():
        print(f"FEHLER: Die Datensatz-Datei '{dataset_yaml_path}' wurde nicht gefunden.")
        print("Bitte stellen Sie sicher, dass das Skript zur Vorbereitung des Bauteil-Datensatzes zuerst ausgeführt wurde.")
        return

    print("Starte das YOLOv11m-Training für den reinen Bauteil-Detektor...")

    # Eindeutiger Name für diesen Trainingslauf
    training_name = 'yolo11m_part_detector_specialist_final'


    model.train(
        data=str(dataset_yaml_path), # Pfad muss als String übergeben werden
        name=training_name,
        epochs=300,
        imgsz=640,
        batch=24,
        patience=30,
        device=device,
        save_period=25 # Speichert alle 25 Epochen einen Sicherheits-Checkpoint
    )

    print("\nTraining des Bauteil-Detektors abgeschlossen!")

    # --- Sicheres Kopieren der Ergebnisse nach Google Drive ---
    print("\nKopiere den gesamten Ergebnisordner nach Google Drive...")
    best_model_path_on_drive = None
    try:
        local_run_directory = model.trainer.save_dir
        drive_dest_directory = GDRIVE_PROJECT_PATH / 'runs/detect' / local_run_directory.name

        if drive_dest_directory.exists():
            print(f"Zielordner '{drive_dest_directory}' existiert bereits. Lösche ihn für eine saubere Kopie.")
            shutil.rmtree(drive_dest_directory)

        shutil.copytree(local_run_directory, drive_dest_directory)

        best_model_path_on_drive = drive_dest_directory / 'weights' / 'best.pt'
        print(f"Erfolgreich nach '{drive_dest_directory}' kopiert.")

    except Exception as e:
        print(f"FEHLER beim Kopieren der Ergebnisse: {e}")


    # --- Finaler Test auf dem separaten Test-Set ---
    if best_model_path_on_drive and best_model_path_on_drive.exists():
        print(f"\nStarte finale Evaluierung mit dem besten Modell auf dem Test-Set...")

        # Lade das gerade gesicherte, beste Modell von Google Drive
        final_model = YOLO(best_model_path_on_drive)

        # Führe die Validierung auf dem Test-Set aus.
        results = final_model.val(
            data=str(dataset_yaml_path),
            split='test',
            imgsz=640
        )
        print("\nErgebnisse der finalen Test-Evaluierung wurden angezeigt.")
    else:
        print("\nSkippe finale Evaluierung, da das beste Modell nicht gefunden wurde.")

    print("\n" + "="*50)
    if best_model_path_on_drive and best_model_path_on_drive.exists():
        print(f"Das beste Modell (Stufe 1) ist jetzt sicher auf Google Drive unter:")
        print(f"{best_model_path_on_drive}")
    else:
         print("ACHTUNG: Das finale Modell konnte nicht auf Google Drive gesichert werden.")
    print("="*50)


if __name__ == '__main__':
    train_yolov11_part_detector()


Verwende Gerät: cuda


100%|██████████| 38.8M/38.8M [00:00<00:00, 217MB/s] 


Starte das YOLOv11m-Training für den reinen Bauteil-Detektor...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/YOLO_Car_Damage_Project/datasets/yolo_parts_only_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11m_part_d

100%|██████████| 755k/755k [00:00<00:00, 13.2MB/s]

Overriding model.yaml nc=80 with nc=21

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytic

...

                   all        149       2456      0.914       0.87      0.911      0.718
            Windshield         90         90      0.977      0.989      0.994       0.91
       Back-windshield         45         45      0.917      0.978      0.986      0.912
          Front-window        131        151      0.908      0.887      0.943      0.647
           Back-window        130        200      0.887      0.864      0.915      0.655
            Front-door        139        139          1      0.991      0.995      0.891
             Back-door        110        110      0.952      0.982      0.979       0.89
           Front-wheel        140        141      0.989      0.957      0.982      0.847
            Back-wheel        133        133      0.957      0.985      0.988      0.847
          Front-bumper        103        106      0.968      0.887      0.942      0.779
           Back-bumper         71         72      0.912      0.792      0.877      0.705
             Headligh

val: Scanning /content/drive/MyDrive/YOLO_Car_Damage_Project/datasets/yolo_parts_only_dataset/labels/test... 149 images, 0 backgrounds, 0 corrupt: 100%|██████████| 149/149 [00:00<00:00, 309.46it/s]


val: New cache created: /content/drive/MyDrive/YOLO_Car_Damage_Project/datasets/yolo_parts_only_dataset/labels/test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.40it/s]


                   all        149       2422      0.896      0.862      0.903      0.712
            Windshield        104        105       0.99      0.942      0.987      0.922
       Back-windshield         35         35      0.962      0.943      0.988      0.838
          Front-window        126        143      0.855      0.888      0.904      0.682
           Back-window        120        178      0.863      0.899      0.918      0.657
            Front-door        139        144      0.981      0.924      0.965      0.861
             Back-door        100        102      0.934      0.951      0.983      0.877
           Front-wheel        138        140      0.985      0.961      0.983       0.86
            Back-wheel        126        126      0.979      0.976      0.989      0.797
          Front-bumper        118        120      0.975      0.917      0.958      0.797
           Back-bumper         52         52        0.9      0.885      0.952      0.752
             Headligh

# Stufe 2 Modell: Zuschneiden der Bauteile für das Training eines 2. Modells nur auf Bildern der beschädigten Bauteile

In [ ]:

"""
Finales Skript zur Erstellung des vollständigen Stufe-2-Trainingsdatensatzes
für die Schadenssegmentierung mit einer 70/15/15 Aufteilung (Train/Val/Test).

Diese Version fasst Klassen zusammen und lädt den Datensatz automatisch herunter,
um Pfad-Fehler zu vermeiden.
"""

import cv2
import json
from pathlib import Path
import shutil
import random
import numpy as np
import yaml
import kagglehub
from ultralytics import YOLO
import torch
from tqdm import tqdm


PART_DETECTOR_MODEL_PATH = Path('/content/drive/MyDrive/YOLO_Car_Damage_Project/runs/detect/yolo11m_part_detector_specialist_final/weights/best.pt')

# Name des finalen Ausgabeordners
OUTPUT_DATASET_DIR = Path('./yolo_damage_segmentation_dataset_stage2_final')

# Konfidenzschwelle für das Bauteil-Modell beim Pseudo-Labeling
CONFIDENCE_THRESHOLD = 0.5

# --- Aufteilung für Trainings-, Validierungs- UND Test-Daten ---
# 70% Training, 15% Validierung, 15% Test
VALIDATION_SPLIT = 0.15
TEST_SPLIT = 0.15


# Globale Variablen für Pfade, die in main() gesetzt werden
parts_img_dir, parts_ann_dir = None, None
damages_img_dir, damages_ann_dir = None, None


# 'Paint chip' und 'Flaking' werden zu 'Paint_Damage' zusammengefasst.
DAMAGE_CLASSES = sorted([
    'Dent', 'Cracked', 'Scratch', 'Paint_Damage', 'Broken part',
    'Missing part'
])
CLASS_TO_IDX = {name: i for i, name in enumerate(DAMAGE_CLASSES)}


def do_boxes_intersect(boxA, boxB):
    """Prüft, ob sich zwei Bounding Boxes (Format [x1, y1, x2, y2]) überschneiden."""
    if boxA[0] > boxB[2] or boxA[2] < boxB[0] or boxA[1] > boxB[3] or boxA[3] < boxB[1]:
        return False
    return True

def clip_and_recalculate_polygon(damage_poly, part_box_coords, original_shape):
    """
    Schneidet ein Polygon an den Rändern einer Bounding Box ab, rechnet die Koordinaten um
    und stellt sicher, dass das resultierende Polygon valide ist.
    """
    crop_x1, crop_y1, crop_x2, crop_y2 = part_box_coords

    if crop_x1 >= crop_x2 or crop_y1 >= crop_y2:
        return None

    crop_w = crop_x2 - crop_x1
    crop_h = crop_y2 - crop_y1

    mask = np.zeros(original_shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [damage_poly], 255)

    cropped_mask = mask[crop_y1:crop_y2, crop_x1:crop_x2]

    contours, _ = cv2.findContours(cropped_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    largest_contour = max(contours, key=cv2.contourArea)

    if len(largest_contour) < 3:
        return None

    yolo_points = []
    for point in largest_contour:
        x, y = point[0]
        norm_x = x / crop_w
        norm_y = y / crop_h
        yolo_points.extend([f"{norm_x:.6f}", f"{norm_y:.6f}"])

    return " ".join(yolo_points)


def process_batch(image_names, source_type, model, all_image_paths, temp_dir):
    """Verarbeitet eine Liste von Bildern, entweder mit echten oder Pseudo-Labels."""
    processed_files = []

    for name in tqdm(image_names, desc=f"Verarbeite {source_type}"):
        img_path = all_image_paths.get(name)
        if not img_path or not img_path.exists(): continue

        original_image = cv2.imread(str(img_path))
        if original_image is None: continue

        damage_ann_path = damages_ann_dir / f"{name}.json"
        try:
            with open(damage_ann_path, 'r', encoding='utf-8') as f:
                damages_data = json.load(f)
        except Exception:
            continue

        all_damages = []
        for dmg_obj in damages_data.get('objects', []):
            original_class = dmg_obj.get('classTitle')
            if original_class in ['Paint chip', 'Flaking']:
                processed_class = 'Paint_Damage'
            elif original_class in DAMAGE_CLASSES:
                processed_class = original_class
            else:
                continue
            if dmg_obj.get('geometryType') == 'polygon':
                poly_points = np.array([(p[0], p[1]) for p in dmg_obj.get("points", {}).get("exterior", [])], dtype=np.int32)
                if len(poly_points) >= 3:
                    x, y, w, h = cv2.boundingRect(poly_points)
                    damage_box = [x, y, x + w, y + h]
                    all_damages.append({'obj': dmg_obj, 'box': damage_box, 'poly': poly_points, 'processed_class': processed_class})

        part_boxes_to_process = []
        # --- KORREKTUR ---
        # Der Name hier muss mit dem Aufruf aus main() übereinstimmen.
        if source_type == "Standard-Labeling":
            part_ann_path = parts_ann_dir / f"{name}.json"
            try:
                with open(part_ann_path, 'r', encoding='utf-8') as f:
                    parts_data = json.load(f)
                for part_obj in parts_data.get('objects', []):
                    if part_obj.get('geometryType') == 'polygon':
                        part_poly = np.array([(p[0], p[1]) for p in part_obj.get("points", {}).get("exterior", [])], dtype=np.int32)
                        if len(part_poly) >= 3:
                            x, y, w, h = cv2.boundingRect(part_poly)
                            part_boxes_to_process.append({'box': [x, y, x + w, y + h], 'name': part_obj['classTitle']})
            except Exception:
                continue
        else: # Pseudo-Labeling
            results = model(original_image, conf=CONFIDENCE_THRESHOLD, verbose=False)
            for res in results:
                for i, box in enumerate(res.boxes.xyxy.cpu().numpy()):
                    part_boxes_to_process.append({'box': box, 'name': res.names[int(res.boxes.cls[i])]})

        part_counter = 0
        for part in part_boxes_to_process:
            part_box = part['box']
            damages_in_this_part = [dmg for dmg in all_damages if do_boxes_intersect(part_box, dmg['box'])]
            if damages_in_this_part:
                crop_x1, crop_y1, crop_x2, crop_y2 = map(int, part_box)
                if crop_x1 >= crop_x2 or crop_y1 >= crop_y2: continue
                cropped_image = original_image[crop_y1:crop_y2, crop_x1:crop_x2]
                if cropped_image.size == 0: continue
                yolo_annotations = []
                for damage in damages_in_this_part:
                    clipped_poly_str = clip_and_recalculate_polygon(damage['poly'], (crop_x1, crop_y1, crop_x2, crop_y2), original_image.shape)
                    if clipped_poly_str:
                        class_idx = CLASS_TO_IDX[damage['processed_class']]
                        yolo_annotations.append(f"{class_idx} {clipped_poly_str}")
                if yolo_annotations:
                    part_name = part['name'].replace(' ', '_')
                    base_name = f"{Path(name).stem}_{part_name}_{part_counter}"
                    new_img_path = temp_dir / 'images' / f"{base_name}.jpg"
                    new_label_path = temp_dir / 'labels' / f"{base_name}.txt"
                    cv2.imwrite(str(new_img_path), cropped_image)
                    with open(new_label_path, 'w', encoding='utf-8') as f:
                        f.write("\n".join(yolo_annotations))
                    processed_files.append((new_img_path, new_label_path))
                    part_counter += 1
    return processed_files


def main():
    """Hauptfunktion zur Ausführung aller Vorverarbeitungsschritte."""
    print("Starte Erstellung des finalen Stufe-2-Datensatzes (mit 70/15/15 Split)...")

    try:
        print("Lade Datensatz von Kaggle Hub herunter...")
        # kagglehub.login() # Ggf. einkommentieren, falls Authentifizierung nötig ist
        download_path = Path(kagglehub.dataset_download("humansintheloop/car-parts-and-car-damages"))
        print(f"Datensatz erfolgreich nach '{download_path}' heruntergeladen.")
        BASE_DATASET_PATH = download_path
    except Exception as e:
        print(f"FEHLER beim Herunterladen des Datensatzes: {e}")
        return

    # Überprüfe den Pfad zum vortrainierten Modell
    if not PART_DETECTOR_MODEL_PATH.exists():
        print(f"FEHLER: Das Bauteil-Modell wurde nicht unter '{PART_DETECTOR_MODEL_PATH}' gefunden.")
        return

    # Globale Pfadvariablen setzen
    global parts_img_dir, parts_ann_dir, damages_img_dir, damages_ann_dir
    PARTS_DATA_PATH = BASE_DATASET_PATH / "Car damages dataset" / "File1"
    parts_img_dir = PARTS_DATA_PATH / "img"
    parts_ann_dir = PARTS_DATA_PATH / "ann"
    DAMAGES_DATA_PATH = BASE_DATASET_PATH / "Car parts dataset" / "File1"
    damages_img_dir = DAMAGES_DATA_PATH / "img"
    damages_ann_dir = DAMAGES_DATA_PATH / "ann"


    # Temporären und finalen Ausgabeordner vorbereiten
    if OUTPUT_DATASET_DIR.exists(): shutil.rmtree(OUTPUT_DATASET_DIR)
    temp_dir = Path('./temp_dataset_build')
    if temp_dir.exists(): shutil.rmtree(temp_dir)
    (temp_dir / 'images').mkdir(parents=True)
    (temp_dir / 'labels').mkdir(parents=True)

    # Sammle alle Bildpfade und Annotationen
    all_image_paths = {p.name: p for p in list(parts_img_dir.glob('*.*')) + list(damages_img_dir.glob('*.*'))}
    damage_ann_stems = {p.stem for p in damages_ann_dir.glob('*.json')}
    part_ann_stems = {p.stem for p in parts_ann_dir.glob('*.json')}

    # Filtere Bilder, die eine Schadens-Annotation haben
    all_unique_image_names = sorted([name for name in all_image_paths if name in damage_ann_stems])

    # Teile in Standard-Labeling und Pseudo-Labeling auf
    overlapping_names = [name for name in all_unique_image_names if name in part_ann_stems]
    damages_only_names = [name for name in all_unique_image_names if name not in part_ann_stems]

    print(f"\nBilder mit Schadens-Annotationen, die für die Verarbeitung ausgewählt wurden: {len(all_unique_image_names)}")
    print(f"Standard-Labeling Bilder (manuelle Bauteile): {len(overlapping_names)}")
    print(f"Pseudo-Labeling Bilder (vorhergesagte Bauteile): {len(damages_only_names)}\n")

    if not all_unique_image_names:
        print("WARNUNG: Keine Bilder mit passenden Schadens-Annotationen gefunden. Überprüfen Sie die Pfade und die Dateistruktur.")
        return

    # Verarbeite die Daten
    all_created_files = process_batch(overlapping_names, "Standard-Labeling", None, all_image_paths, temp_dir)
    if damages_only_names:
        print("\nLade Bauteil-Detektor-Modell...")
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        part_model = YOLO(PART_DETECTOR_MODEL_PATH)
        part_model.to(device)
        all_created_files.extend(process_batch(damages_only_names, "Pseudo-Labeling", part_model, all_image_paths, temp_dir))

    print(f"\nInsgesamt {len(all_created_files)} zugeschnittene Bilder mit Labels erstellt.")
    if not all_created_files:
        print("\nWARNUNG: Keine Bilder wurden verarbeitet. Der Datensatz ist leer.")
        shutil.rmtree(temp_dir)
        return

    # Aufteilung in Train, Val UND Test Sets
    train_percentage = 1 - VALIDATION_SPLIT - TEST_SPLIT
    print(f"\nAufteilen in Trainings- ({train_percentage:.0%}), Validierungs- ({VALIDATION_SPLIT:.0%}) und Test-Sets ({TEST_SPLIT:.0%})...")
    random.shuffle(all_created_files)

    val_split_idx = int(len(all_created_files) * VALIDATION_SPLIT)
    test_split_idx = val_split_idx + int(len(all_created_files) * TEST_SPLIT)

    val_files = all_created_files[:val_split_idx]
    test_files = all_created_files[val_split_idx:test_split_idx]
    train_files = all_created_files[test_split_idx:]

    # Alle drei Sets verschieben
    for subset_name, files in [('train', train_files), ('val', val_files), ('test', test_files)]:
        (OUTPUT_DATASET_DIR / 'images' / subset_name).mkdir(parents=True)
        (OUTPUT_DATASET_DIR / 'labels' / subset_name).mkdir(parents=True)
        for img_file, label_file in files:
            shutil.move(str(img_file), str(OUTPUT_DATASET_DIR / 'images' / subset_name / img_file.name))
            shutil.move(str(label_file), str(OUTPUT_DATASET_DIR / 'labels' / subset_name / label_file.name))

    shutil.rmtree(temp_dir)

    # Finale dataset.yaml mit Test-Set
    yaml_data = {
        'path': str(OUTPUT_DATASET_DIR.resolve()),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(DAMAGE_CLASSES),
        'names': {i: name for i, name in enumerate(DAMAGE_CLASSES)}
    }
    with open(OUTPUT_DATASET_DIR / 'dataset.yaml', 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)

    print("\n---Prozess Abgeschlossen ---")
    print(f"Finaler Datensatz für Stufe 2 wurde in '{OUTPUT_DATASET_DIR.resolve()}' erstellt.")
    print(f"Trainingsbilder: {len(train_files)}")
    print(f"Validierungsbilder: {len(val_files)}")
    print(f"Testbilder: {len(test_files)}")


if __name__ == '__main__':
    main()


Starte Erstellung des finalen Stufe-2-Datensatzes (mit 70/15/15 Split)...
Lade Datensatz von Kaggle Hub herunter...


100%|██████████| 2.89G/2.89G [01:41<00:00, 30.5MB/s]

Extracting files...


Datensatz erfolgreich nach '/root/.cache/kagglehub/datasets/humansintheloop/car-parts-and-car-damages/versions/2' heruntergeladen.

Bilder mit Schadens-Annotationen, die für die Verarbeitung ausgewählt wurden: 814
Standard-Labeling Bilder (manuelle Bauteile): 441
Pseudo-Labeling Bilder (vorhergesagte Bauteile): 373



Verarbeite Standard-Labeling: 100%|██████████| 441/441 [00:08<00:00, 49.98it/s]



Lade Bauteil-Detektor-Modell...


Verarbeite Pseudo-Labeling: 100%|██████████| 373/373 [00:16<00:00, 23.18it/s]



Insgesamt 6082 zugeschnittene Bilder mit Labels erstellt.

Aufteilen in Trainings- (70%), Validierungs- (15%) und Test-Sets (15%)...

---Prozess Abgeschlossen ---
Finaler Datensatz für Stufe 2 wurde in '/content/yolo_damage_segmentation_dataset_stage2_final' erstellt.
Trainingsbilder: 4258
Validierungsbilder: 912
Testbilder: 912


# Training des Stufe 2 Modells

In [ ]:

"""
FINALES Trainings-Skript für den Stufe-2 Schadens-Segmentierer.

"""
import os
import sys
import subprocess
import pkg_resources
import torch
from pathlib import Path
import shutil
from ultralytics import YOLO

# --- Pfad zum Haupt-Projektordner auf Google Drive ---
GDRIVE_PROJECT_PATH = Path('/content/drive/MyDrive/YOLO_Car_Damage_Project')


def train_final_damage_segmenter():
    """
    Trainiert, speichert und evaluiert das finale YOLO-Modell.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Verwende Gerät: {device}")

    if not GDRIVE_PROJECT_PATH.exists():
        print(f"FEHLER: Der Google Drive Projektordner '{GDRIVE_PROJECT_PATH}' wurde nicht gefunden.")
        return

    model = YOLO('yolo11l-seg.pt')
    dataset_yaml_path = './yolo_damage_segmentation_dataset_stage2_final/dataset.yaml'

    if not Path(dataset_yaml_path).exists():
        print(f"FEHLER: Die Datensatz-Datei '{dataset_yaml_path}' wurde nicht gefunden.")
        return

    print("\nStarte das FINALE YOLO-Segmentierungs-Training...")

    training_name = 'yolo11l_damage_segmenter_1024_stage2_FINAL'


    model.train(
        data=dataset_yaml_path,
        name=training_name,
        epochs=220,
        imgsz=1024, # höhere Auflösung für feine Kratzer und Lackschäden
        batch=16,
        patience=30,
        device=device,
        save_period=25
    )

    print("\nTraining abgeschlossen!")

    # --- Sicheres Kopieren der Ergebnisse nach Google Drive ---
    print("\nKopiere den gesamten Ergebnisordner nach Google Drive...")
    best_model_path_on_drive = None
    try:
        # Pfad zum lokalen Ordner, in dem das Training gespeichert wurde
        local_run_directory = model.trainer.save_dir
        # Zielordner auf Google Drive
        drive_dest_directory = GDRIVE_PROJECT_PATH / 'runs/segment' / local_run_directory.name

        if drive_dest_directory.exists():
            print(f"Zielordner '{drive_dest_directory}' existiert bereits. Lösche ihn für eine saubere Kopie.")
            shutil.rmtree(drive_dest_directory)

        # Kopiere den gesamten Ordner in einem Rutsch
        shutil.copytree(local_run_directory, drive_dest_directory)

        best_model_path_on_drive = drive_dest_directory / 'weights' / 'best.pt'
        print(f"Erfolgreich nach '{drive_dest_directory}' kopiert.")

    except Exception as e:
        print(f"FEHLER beim Kopieren der Ergebnisse: {e}")

    # --- Finaler Test auf dem Test-Set ---
    if best_model_path_on_drive and best_model_path_on_drive.exists():
        print(f"\nStarte finale Evaluierung mit dem Modell von Google Drive...")

        final_model = YOLO(best_model_path_on_drive)

        results = final_model.val(
            data=dataset_yaml_path,
            split='test',
            imgsz=1024
        )
        print("\nErgebnisse der finalen Test-Evaluierung wurden angezeigt.")
    else:
        print("\nSkippe finale Evaluierung, da das beste Modell nicht gefunden wurde.")

    print("\n" + "="*50)
    if best_model_path_on_drive and best_model_path_on_drive.exists():
        print(f"Das finale, beste Modell ist jetzt sicher auf Google Drive unter:")
        print(f"{best_model_path_on_drive}")
    else:
        print("ACHTUNG: Das finale Modell konnte nicht auf Google Drive gesichert werden.")
    print("="*50)


if __name__ == '__main__':
    train_final_damage_segmenter()


Verwende Gerät: cuda

Starte das FINALE YOLO-Segmentierungs-Training...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./yolo_damage_segmentation_dataset_stage2_final/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=220, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11l_damage_segmenter_1024_sta

train: Scanning /content/yolo_damage_segmentation_dataset_stage2_final/labels/train.cache... 4253 images, 0 backgrounds, 5 corrupt: 100%|██████████| 4258/4258 [00:00<?, ?it/s]

train: /content/yolo_damage_segmentation_dataset_stage2_final/images/train/Car damages 101_Headlight_2.jpg: ignoring corrupt image/label: image size (33, 8) <10 pixels
train: /content/yolo_damage_segmentation_dataset_stage2_final/images/train/Car damages 236_Grille_2.jpg: 1 duplicate labels removed
train: /content/yolo_damage_segmentation_dataset_stage2_final/images/train/Car damages 500_Back-window_8.jpg: ignoring corrupt image/label: image size (33, 8) <10 pixels
train: /content/yolo_damage_segmentation_dataset_stage2_final/images/train/Car damages 636_Headlight_6.jpg: 1 duplicate labels removed
train: /content/yolo_damage_segmentation_dataset_stage2_final/images/train/Car damages 698_Tail-light_10.jpg: ignoring corrupt image/label: image size (15, 6) <10 pixels
train: /content/yolo_damage_segmentation_dataset_stage2_final/images/train/Car damages 796_Back-window_13.jpg: ignoring corrupt image/label: image size (19, 9) <10 pixels
train: /content/yolo_damage_segmentation_dataset_stage

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 548.0±342.3 MB/s, size: 24.6 KB)


val: Scanning /content/yolo_damage_segmentation_dataset_stage2_final/labels/val.cache... 910 images, 0 backgrounds, 2 corrupt: 100%|██████████| 912/912 [00:00<?, ?it/s]

...

val: /content/yolo_damage_segmentation_dataset_stage2_final/images/test/Car damages 488_Headlight_7.jpg: ignoring corrupt image/label: image size (38, 8) <10 pixels
val: /content/yolo_damage_segmentation_dataset_stage2_final/images/test/Car damages 766_Mirror_10.jpg: ignoring corrupt image/label: image size (9, 11) <10 pixels
val: /content/yolo_damage_segmentation_dataset_stage2_final/images/test/Car damages 800_Back-bumper_8.jpg: ignoring corrupt image/label: image size (34, 9) <10 pixels
val: /content/yolo_damage_segmentation_dataset_stage2_final/images/test/Car damages 859_Grille_7.jpg: 1 duplicate labels removed
val: New cache created: /content/yolo_damage_segmentation_dataset_stage2_final/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   4%|▎         | 2/57 [00:00<00:20,  2.71it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:12<00:00,  4.63it/s]


                   all        909       3386      0.599      0.372      0.408      0.282      0.565      0.348      0.367      0.207
           Broken part        602        912      0.623      0.467      0.491      0.352       0.56      0.418      0.411      0.221
               Cracked         53         54      0.594      0.325      0.382      0.301      0.592      0.323      0.378      0.288
                  Dent        448        719      0.638      0.343       0.38      0.238      0.576      0.309      0.328      0.166
          Missing part        201        235      0.621      0.419      0.461      0.326      0.621      0.417      0.441      0.263
          Paint_Damage        157        470      0.546       0.34      0.359      0.235      0.516      0.319      0.322      0.157
               Scratch        272        996      0.573      0.336      0.377      0.243      0.526      0.304       0.32      0.146
Speed: 0.4ms preprocess, 7.6ms inference, 0.0ms loss, 1.5ms postproce